In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [5]:
# movies = movies.rename(columns={'id':'movie_id'})
# movies.merge(credits, on='movie_id').shape
movies = movies.merge(credits, on='title')

In [ ]:
movies.info()

In [6]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.shape

(4809, 7)

In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace=True)

In [9]:
movies.duplicated().sum()

0

In [10]:
import ast

In [12]:
print(type(movies.iloc[0].genres))#this is string need to convert it into list
def dect_to_list(obj):
  list1 = []
  for i in ast.literal_eval(obj):
    list1.append(i["name"])
  return list1
dect_to_list(movies.iloc[0].genres)

<class 'str'>


['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [13]:
movies['genres'] = movies['genres'].apply(dect_to_list)

In [14]:
movies['keywords'] = movies['keywords'].apply(dect_to_list)

In [15]:
def eval(obj):
  list1 = []
  cnt = 0
  for i in ast.literal_eval(obj):
    while cnt < 3:
      list1.append(i['name'])
      cnt += 1
  return list1

In [16]:
movies['cast'] = movies['cast'].apply(eval)

In [17]:
def director(obj):
  list1 = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      list1.append(i['name'])
      break
  return list1

In [18]:
movies['crew'] = movies['crew'].apply(director)

In [19]:
movies['overview'].apply(lambda x:x.split())[0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [20]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [21]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [22]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [23]:
new_df = movies[['movie_id','title','tags']]

In [24]:
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [25]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [27]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\nadip\AppData\Local\Temp\ipykernel_7368\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors1 = cv.fit_transform(new_df['tags']).toarray()

In [29]:
cv.get_feature_names_out()
len(cv.get_feature_names_out())

5000

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vectors2 = tfidf.fit_transform(new_df['tags']).toarray()

In [31]:
tfidf.get_feature_names_out()
len(tfidf.get_feature_names_out())

5000

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
similarity1 = cosine_similarity(vectors1)
similarity2 = cosine_similarity(vectors2)

In [34]:
sorted(list(enumerate(similarity2[0])),reverse=True,key=lambda x:x[1])

[(0, 1.0),
 (1147, 0.37156309708659985),
 (132, 0.3327838835762578),
 (206, 0.32864434842754375),
 (2405, 0.16154318471254955),
 (1214, 0.15656780030578818),
 (3606, 0.14912887238262235),
 (582, 0.14412557655592667),
 (3728, 0.13605524638403038),
 (1916, 0.1300832782425503),
 (778, 0.1246497442784093),
 (539, 0.1172554481782738),
 (1202, 0.11574874811164533),
 (507, 0.11552440785299677),
 (3536, 0.11504821521470704),
 (260, 0.11423734353053705),
 (1087, 0.11396503478817385),
 (2782, 0.11326226375147382),
 (373, 0.11167865183636277),
 (4046, 0.11073738328774632),
 (47, 0.1054523723799459),
 (557, 0.10507195660620627),
 (438, 0.103858697178116),
 (74, 0.10330583068208532),
 (61, 0.10325470266048772),
 (3671, 0.10173025187065332),
 (311, 0.10172383181319351),
 (184, 0.10039775276301424),
 (4190, 0.09982831550906204),
 (4, 0.09570884986040604),
 (1273, 0.0953215396738788),
 (942, 0.0951076879797596),
 (2969, 0.09502479173199299),
 (513, 0.09495189167203993),
 (220, 0.09486345367870312),
 (

In [35]:
def recommend1(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity1[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [36]:
def recommend2(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity2[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [37]:
def recommend_multi(movies):
    # Get indices for each input movie
    movie_indices = [new_df[new_df['title'] == movie].index[0] for movie in movies if movie in new_df['title'].values]
    
    if not movie_indices:
        print("None of the movies were found.")
        return

    # Sum their similarity vectors
    combined_similarity = sum(similarity2[i] for i in movie_indices) / len(movie_indices)

    # Enumerate and sort (excluding input movies)
    movies_list = sorted(
        list(enumerate(combined_similarity)),
        reverse=True,
        key=lambda x: x[1]
    )

    # Remove the input movies from the recommendations
    recommended = []
    for i in movies_list:
        if i[0] not in movie_indices:
            recommended.append(new_df.iloc[i[0]].title)
        if len(recommended) == 5:
            break

    # Print recommendations
    for title in recommended:
        print(title)


In [40]:
recommend1('Avatar')

Aliens vs Predator: Requiem
Man on a Ledge
Clash of the Titans
Jupiter Ascending
Independence Day


In [39]:
recommend2('Batman Begins')

The Dark Knight
The Dark Knight Rises
American Hustle
American Psycho
The Big Short


In [38]:
recommend_multi(["Inception", "The Dark Knight"])

The Dark Knight Rises
Batman Begins
J. Edgar
Shutter Island
Batman Returns
